# Unsupervised Learning Trading Strategy

Your KMeans clustering is totally useless when you're not normalizing the RSI values. In essence all the other features are just small noise in the clustering and you end up making the wrong conclusion that RSI is the main feature driving the clustering in your data.

In [1]:
from statsmodels.regression.rolling import RollingOLS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import datetime as dt
import yfinance as yf
import pandas_datareader.data as web
import pandas_ta
import warnings
warnings.filterwarnings('ignore')
import time


Getting the companies' symbols from Wiki.
Caution : not survivalship bias free

In [2]:
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500.head()


,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [3]:
sp500.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Symbol                 503 non-null    object
 1   Security               503 non-null    object
 2   GICS Sector            503 non-null    object
 3   GICS Sub-Industry      503 non-null    object
 4   Headquarters Location  503 non-null    object
 5   Date added             503 non-null    object
 6   CIK                    503 non-null    int64 
 7   Founded                503 non-null    object
dtypes: int64(1), object(7)
memory usage: 31.6+ KB


replacing `.` with `-`

In [4]:
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')


In [5]:
symbols_list = sp500['Symbol'].unique().tolist()
symbols_list[:10]


['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADM', 'ADBE', 'ADP', 'AES', 'AFL']

defining time limits

In [6]:
end_date = '2023-11-18' #'2023-09-27'
start_date = pd.to_datetime(end_date) - pd.DateOffset(365*8) #'2023-10-27'
start_date


Timestamp('2015-11-20 00:00:00')

In [7]:
original_df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date)

#time.sleep(1)
original_df.head()


[*********************100%***********************]  503 of 503 completed


Adj Close                                        \
                                   A        AAL       AAPL       ABBV ABNB   
Date                                                                         
2015-11-20 00:00:00-05:00  36.774700  40.328243  27.152472  43.255482  NaN   
2015-11-23 00:00:00-05:00  37.476860  40.423801  26.799694  43.283794  NaN   
2015-11-24 00:00:00-05:00  38.076046  39.401268  27.056885  43.269634  NaN   
2015-11-25 00:00:00-05:00  38.703312  39.477711  26.863422  42.675053  NaN   
2015-11-27 00:00:00-05:00  39.293114  39.840858  26.813353  42.469788  NaN   

                                                                       \
                                 ABT       ACGL        ACN       ADBE   
Date                                                                    
2015-11-20 00:00:00-05:00  39.607716  24.686666  95.005859  91.809998   
2015-11-23 00:00:00-05:00  39.117043  24.643333  94.891487  91.959999   
2015-11-24 00:00:00-05:00  39.125645  24.533333  94.460297  92.000000   
2015-11-25 00:00:00-05:00  39.091217  24.216667  94.240334  91.769997   
2015-11-27 00:00:00-05:00  39.099819  24.306667  94.557098  92.169998   

                                      ...   Volume                             \
                                 ADI  ...     WYNN      XEL       XOM    XRAY   
Date                                  ...                                       
2015-11-20 00:00:00-05:00  50.314316  ...  3778300  3883400  10144400  880900   
2015-11-23 00:00:00-05:00  48.095818  ...  2427400  2341500  11868400  616000   
2015-11-24 00:00:00-05:00  51.161068  ...  5628300  4416300  15055500  917900   
2015-11-25 00:00:00-05:00  51.017117  ...  2358500  2200100   8980400  365000   
2015-11-27 00:00:00-05:00  51.186481  ...  2285100  1468000   4156600  293200   

                                                                                
                               XYL      YUM      ZBH    ZBRA     ZION      ZTS  
Date                                                                            
2015-11-20 00:00:00-05:00  1300900  5928998  1692187  303500  2092400  3679400  
2015-11-23 00:00:00-05:00   870600  3554005  1031648  320300  1438700  3265300  
2015-11-24 00:00:00-05:00  1044000  3045455  1313868  431800  1608600  1713500  
2015-11-25 00:00:00-05:00   916200  3222808  1008576  292100  1086200  2196800  
2015-11-27 00:00:00-05:00   514200  2558884   917730  175700   763200  1155100  

[5 rows x 3018 columns]

In [9]:
# getting rid of multi index
df = original_df.stack()
df


Adj Close       Close        High  \
Date                                                                 
2015-11-20 00:00:00-05:00 A      36.774700   39.279999   39.349998   
                          AAL    40.328243   42.200001   42.709999   
                          AAPL   27.152472   29.825001   29.980000   
                          ABBV   43.255482   61.110001   61.360001   
                          ABT    39.607716   46.009998   46.380001   
...                                    ...         ...         ...   
2023-11-17 00:00:00-05:00 YUM   127.660004  127.660004  128.490005   
                          ZBH   111.669998  111.669998  112.660004   
                          ZBRA  218.020004  218.020004  218.699997   
                          ZION   36.070000   36.070000   36.470001   
                          ZTS   174.800003  174.800003  177.699997   

                                       Low        Open       Volume  
Date                                                                 
2015-11-20 00:00:00-05:00 A      38.520000   38.779999    5447900.0  
                          AAL    42.049999   42.400002    5656000.0  
                          AAPL   29.712500   29.799999  137148400.0  
                          ABBV   60.570000   60.939999    7490200.0  
                          ABT    45.840000   46.029999    7140700.0  
...                                    ...         ...          ...  
2023-11-17 00:00:00-05:00 YUM   127.250000  128.419998    1089600.0  
                          ZBH   111.330002  112.349998    2991400.0  
                          ZBRA  215.270004  218.460007     340600.0  
                          ZION   35.400002   36.250000    2136200.0  
                          ZTS   173.699997  177.410004    2425100.0  

[994819 rows x 6 columns]

In [11]:
df.index.names = ['date','ticker']
df.head()


Adj Close      Close       High        Low  \
date                      ticker                                               
2015-11-20 00:00:00-05:00 A       36.774700  39.279999  39.349998  38.520000   
                          AAL     40.328243  42.200001  42.709999  42.049999   
                          AAPL    27.152472  29.825001  29.980000  29.712500   
                          ABBV    43.255482  61.110001  61.360001  60.570000   
                          ABT     39.607716  46.009998  46.380001  45.840000   

                                       Open       Volume  
date                      ticker                          
2015-11-20 00:00:00-05:00 A       38.779999    5447900.0  
                          AAL     42.400002    5656000.0  
                          AAPL    29.799999  137148400.0  
                          ABBV    60.939999    7490200.0  
                          ABT     46.029999    7140700.0

In [12]:
df.columns = df.columns.str.lower()
df.head()


adj close      close       high        low  \
date                      ticker                                               
2015-11-20 00:00:00-05:00 A       36.774700  39.279999  39.349998  38.520000   
                          AAL     40.328243  42.200001  42.709999  42.049999   
                          AAPL    27.152472  29.825001  29.980000  29.712500   
                          ABBV    43.255482  61.110001  61.360001  60.570000   
                          ABT     39.607716  46.009998  46.380001  45.840000   

                                       open       volume  
date                      ticker                          
2015-11-20 00:00:00-05:00 A       38.779999    5447900.0  
                          AAL     42.400002    5656000.0  
                          AAPL    29.799999  137148400.0  
                          ABBV    60.939999    7490200.0  
                          ABT     46.029999    7140700.0

In [13]:
#saving data for futher tests and avoiding being blocked
df.to_csv('assets_stack.csv')


## Calculate features and technical indicators for each stock


* garman-klass volatility
* RSI
* Bollinger Bands
* ATR
* MACD
* Dollar Volume

$$ \sigma_{GK} = \sqrt{\frac{1}{N-1} \sum_{i=1}^{N} \left(\log\left(\frac{H_i}{L_i}\right)^2 - \frac{2\log\left(\frac{C_i}{O_i}\right)\log\left(\frac{H_i}{L_i}\right)}{N} + \log\left(\frac{C_i}{O_i}\right)^2\right)}


* N is the number of data points i
* H\_i  is the high price of the asset at time i
* L\_i is the low price of the asset at time i
* O\_i is the open price of the asset at time i
* C\_i is the close price of the asset at time i

In [42]:
df['garman_klass_vol'] = ((np.log(df.high) - np.log(df.low))**2)/2 - (2*np.log(2)-1)*(np.log(df['adj close']) - np.log(df.open))**2
df


adj close  ...  garman_klass_vol
date                      ticker              ...                  
2015-09-29 00:00:00-04:00 A        31.588039  ...         -0.000854
                          AAL      37.361618  ...         -0.000443
                          AAPL     24.716068  ...         -0.005441
                          ABBV     37.024612  ...         -0.049280
                          ABT      33.807266  ...         -0.008237
...                                      ...  ...               ...
2023-09-26 00:00:00-04:00 YUM     124.010002  ...          0.000053
                          ZBH     112.216316  ...          0.000224
                          ZBRA    223.960007  ...          0.000133
                          ZION     33.581326  ...          0.000292
                          ZTS     176.447540  ...          0.000075

[994088 rows x 7 columns]

In [48]:
df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))


In [51]:
# normalizing indicator to lower/middle/high band
pandas_ta.bbands(close=df.xs('AAPL', level=1)['adj close'], length=20)


,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0
date,,,,,
2015-09-29 00:00:00-04:00,NaN,NaN,NaN,NaN,NaN
2015-09-30 00:00:00-04:00,NaN,NaN,NaN,NaN,NaN
2015-10-01 00:00:00-04:00,NaN,NaN,NaN,NaN,NaN
2015-10-02 00:00:00-04:00,NaN,NaN,NaN,NaN,NaN
2015-10-05 00:00:00-04:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2023-09-20 00:00:00-04:00,170.438581,180.107718,189.776855,10.737060,0.249274
2023-09-21 00:00:00-04:00,169.695552,179.748691,189.801830,11.185772,0.199222
2023-09-22 00:00:00-04:00,169.479507,179.669295,189.859083,11.342826,0.249294


In [55]:
df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=np.log1p(x), length=20).iloc[:,0])
df


IndexingError: Too many indexers

In [ ]:
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=np.log1p(x), length=20)).iloc[:,1]
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=np.log1p(x), length=20)).iloc[:,2]
